In [ ]:
from google.colab import userdata

DB_USER = userdata.get("DB_USER")
DB_PASS = userdata.get("DB_PASS")
DB_HOST = userdata.get("DB_HOST") or "localhost"
DB_PORT = 3306
DB_NAME = userdata.get("DB_NAME")
NVIDIA_API_KEY = userdata.get("NVIDIA_API_KEY")


In [ ]:
!pip install langchain_nvidia_ai_endpoints
!pip install mysql-connector-python
!pip install rapidfuzz
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 48.7 MB/s eta 0:00:00


In [ ]:
# # ✅ END-TO-END AGENT FOR ENTITY EXTRACTION + ID RESOLUTION WITH IMPROVED FUZZY MATCHING

# from langchain.chains import LLMChain
# from langchain.prompts import PromptTemplate
# from langchain_nvidia_ai_endpoints import ChatNVIDIA
# import mysql.connector
# import os
# import json
# from rapidfuzz import process, fuzz

# # === 1. SETUP NVIDIA LLM ===
# llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1", api_key=NVIDIA_API_KEY)

# # === 2. ENTITY EXTRACTION PROMPT ===
# entity_extraction_prompt = PromptTemplate(
#     input_variables=["query"],
#     template="""
# You are a football data assistant. Extract and classify entities from the user's query into JSON.

# Return this structure:
# {{
#   "player": ...,
#   "team": ...,
#   "league": ...,
#   "season": ...,
#   "event_type": ...,
#   "referee": ...,
#   "round": ...,
#   "venue": ...
# }}

# Leave fields as null if not mentioned.

# User query: "{query}"
# """
# )

# # === 3. ENTITY EXTRACTION CHAIN ===
# entity_chain = LLMChain(llm=llm, prompt=entity_extraction_prompt, verbose=True)

# def extract_entities(user_query: str) -> dict:
#     response = entity_chain.run({"query": user_query})
#     try:
#         return json.loads(response)
#     except:
#         print("⚠️ Failed to parse entity JSON")
#         return {}


# # === 5. FUZZY MATCHING LOOKUP WITH TOKEN_SORT_RATIO ===
# def fuzzy_lookup(query_name: str, table: str, column: str, id_column: str, threshold: int = 75) -> int:
#     try:
#         conn = mysql.connector.connect(
#             host=DB_HOST, user=DB_USER, password=DB_PASS, database=DB_NAME, port=DB_PORT
#         )
#         cursor = conn.cursor()
#         cursor.execute(f"SELECT {id_column}, {column} FROM {table}")
#         rows = cursor.fetchall()
#         conn.close()

#         name_dict = {row[1]: row[0] for row in rows if row[1]}
#         best_match = process.extractOne(
#             query_name,
#             name_dict.keys(),
#             scorer=fuzz.token_sort_ratio
#         )

#         if best_match and best_match[1] >= threshold:
#             print(f"✅ Fuzzy matched '{query_name}' → '{best_match[0]}' ({best_match[1]}%)")
#             return name_dict[best_match[0]]
#         else:
#             print(f"⚠️ No good fuzzy match for '{query_name}' (best: {best_match})")
#             return None
#     except Exception as e:
#         print(f"❌ Error in fuzzy lookup for '{query_name}': {str(e)}")
#         return None

# # === 6. ID FUNCTIONS USING FUZZY MATCHING ===
# def get_player_id(name: str) -> int:
#     return fuzzy_lookup(name, "base_data_apis.players", "name", "player_id")

# def get_team_id(name: str) -> int:
#     return fuzzy_lookup(name, "base_data_apis.teams", "name", "team_id")

# def get_league_id(name: str) -> int:
#     return fuzzy_lookup(name, "base_data_apis.leagues", "name", "league_id")

# # === 7. MAIN FUNCTION: QUERY -> ENTITY -> IDs ===
# def resolve_entities_to_ids(user_query: str) -> dict:
#     entities = extract_entities(user_query)
#     resolved = {}

#     if entities.get("player"):
#         resolved["player_id"] = get_player_id(entities["player"])

#     if entities.get("team"):
#         resolved["team_id"] = get_team_id(entities["team"])

#     if entities.get("league"):
#         league_name = entities["league"]
#         resolved["league_id"] = (
#             39 if league_name.lower() == "premier league" else get_league_id(league_name)
#         )

#     if entities.get("season"):
#         try:
#             resolved["season_year"] = int(entities["season"])
#         except:
#             resolved["season_year"] = None

#     if entities.get("event_type"):
#         resolved["event_type"] = entities["event_type"].capitalize()

#     resolved["raw_entities"] = entities  # for debugging
#     return resolved

# # === 8. USAGE EXAMPLE ===
# if __name__ == "__main__":
#     user_query = "How many goals did Erling Haaland score against Manchester United in Premier League in 2023?"
#     result = resolve_entities_to_ids(user_query)
#     print("\n🎯 Resolved Entity → ID Mapping:")
#     print(json.dumps(result, indent=2))

<ipython-input-6-3353493321>:39: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  entity_chain = LLMChain(llm=llm, prompt=entity_extraction_prompt, verbose=True)
<ipython-input-6-3353493321>:42: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = entity_chain.run({"query": user_query})




> Entering new LLMChain chain...
Prompt after formatting:

You are a football data assistant. Extract and classify entities from the user's query into JSON.

Return this structure:
{
  "player": ...,
  "team": ...,
  "league": ...,
  "season": ...,
  "event_type": ...,
  "referee": ...,
  "round": ...,
  "venue": ...
}

Leave fields as null if not mentioned.

User query: "How many goals did Erling Haaland score against Manchester United in Premier League in 2023?"


> Finished chain.
✅ Fuzzy matched 'Erling Haaland' → 'E. Haaland' (75.0%)
✅ Fuzzy matched 'Manchester United' → 'Manchester United' (100.0%)

🎯 Resolved Entity → ID Mapping:
{
  "player_id": 1100,
  "team_id": 33,
  "league_id": 39,
  "season_year": 2023,
  "event_type": "Goals",
  "raw_entities": {
    "player": "Erling Haaland",
    "team": "Manchester United",
    "league": "Premier League",
    "season": "2023",
    "event_type": "goals",
    "referee": null,
    "round": null,
    "venue": null
  }
}


In [ ]:
# # ✅ END-TO-END AGENT FOR ENTITY EXTRACTION + ID RESOLUTION + DYNAMIC SQL QUERY EXECUTION

# from langchain.chains import LLMChain
# from langchain.prompts import PromptTemplate
# from langchain_nvidia_ai_endpoints import ChatNVIDIA
# import mysql.connector
# import os
# import json
# from rapidfuzz import process, fuzz

# # === 1. SETUP NVIDIA LLM ===
# llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1", api_key=NVIDIA_API_KEY)

# # === 2. ENTITY EXTRACTION PROMPT ===
# entity_extraction_prompt = PromptTemplate(
#     input_variables=["query"],
#     template="""
# You are a football data assistant. Extract and classify entities from the user's query into this JSON:

# {{
#   "player": <player_name or null>,
#   "team": <team_name or null>,
#   "team_role": "for" | "against" | null,
#   "league": <league_name or null>,
#   "season": <season_year or null>,
#   "event_type": {{
#     "type": <Goal|Card|subst|Var or null>,
#     "detail": <valid detail string or null>
#   }},
#   "referee": <referee_name or null>,
#   "round": <round number or null>,
#   "venue": <venue name or null>
# }}

# Valid event type + detail combinations include:
# Goal → Penalty, Normal Goal, Own Goal, Missed Penalty
# Card → Yellow Card, Red Card, (empty)
# subst → Substitution (covers all numbered substitutions)
# Var → Goal cancelled, Penalty cancelled, Card upgrade, Goal confirmed, Card reviewed, Penalty awarded, Penalty confirmed, Red card cancelled, Goal Disallowed, Goal Disallowed - offside, Goal Disallowed - handball, Goal Disallowed - Foul, (empty)

# Always normalize substitution details to just "Substitution"

# User query: "{query}"
# """
# )

# # === 3. ENTITY EXTRACTION CHAIN ===
# entity_chain = LLMChain(llm=llm, prompt=entity_extraction_prompt, verbose=True)

# def extract_entities(user_query: str) -> dict:
#     try:
#         response = entity_chain.run(query=user_query)
#         return json.loads(response)
#     except Exception as e:
#         print("\u26a0\ufe0f Failed to parse entity JSON:", str(e))
#         return {}



# # === 5. FUZZY MATCHING LOOKUP ===
# def fuzzy_lookup(query_name: str, table: str, column: str, id_column: str, threshold: int = 75) -> int:
#     try:
#         conn = mysql.connector.connect(
#             host=DB_HOST, user=DB_USER, password=DB_PASS, database=DB_NAME, port=DB_PORT
#         )
#         cursor = conn.cursor()
#         cursor.execute(f"SELECT {id_column}, {column} FROM {table}")
#         rows = cursor.fetchall()
#         conn.close()

#         name_dict = {row[1]: row[0] for row in rows if row[1]}
#         best_match = process.extractOne(query_name, name_dict.keys(), scorer=fuzz.token_sort_ratio)

#         if best_match and best_match[1] >= threshold:
#             print(f"✅ Fuzzy matched '{query_name}' → '{best_match[0]}' ({best_match[1]}%)")
#             return name_dict[best_match[0]]
#         else:
#             print(f"⚠️ No good fuzzy match for '{query_name}' (best: {best_match})")
#             return None
#     except Exception as e:
#         print(f"❌ Error in fuzzy lookup for '{query_name}': {str(e)}")
#         return None

# # === 6. ENTITY -> ID FUNCTIONS ===
# def get_player_id(name: str) -> int:
#     return fuzzy_lookup(name, "base_data_apis.players", "name", "player_id")

# def get_team_id(name: str) -> int:
#     return fuzzy_lookup(name, "base_data_apis.teams", "name", "team_id")

# def get_league_id(name: str) -> int:
#     return fuzzy_lookup(name, "base_data_apis.leagues", "name", "league_id")

# # === 7. VALID EVENT TYPES ===
# VALID_EVENT_TYPES = {
#     ("Goal", "Penalty"), ("Goal", "Normal Goal"), ("Goal", "Own Goal"), ("Goal", "Missed Penalty"),
#     ("Card", "Yellow Card"), ("Card", "Red Card"), ("Card", ""),
#     ("Var", "Goal cancelled"), ("Var", "Penalty cancelled"), ("Var", "Card upgrade"), ("Var", "Goal confirmed"),
#     ("Var", "Card reviewed"), ("Var", "Penalty awarded"), ("Var", "Penalty confirmed"), ("Var", "Red card cancelled"),
#     ("Var", "Goal Disallowed"), ("Var", "Goal Disallowed - offside"), ("Var", "Goal Disallowed - handball"),
#     ("Var", "Goal Disallowed - Foul"), ("Var", ""),
#     ("subst", "Substitution")
# }

# def normalize_event_type_pair(event_type: str, event_detail: str) -> tuple:
#     if event_type == "subst" and event_detail and event_detail.lower().startswith("substitution"):
#         return ("subst", "Substitution")
#     return (event_type or "", event_detail or "")

# # === 8. MAIN AGENT FUNCTION ===
# def resolve_entities_to_ids(user_query: str) -> dict:
#     entities = extract_entities(user_query)
#     resolved = {}

#     if entities.get("player"):
#         resolved["player_id"] = get_player_id(entities["player"])

#     if entities.get("team"):
#         team_id = get_team_id(entities["team"])
#         role = entities.get("team_role", "for")
#         if role == "for":
#             resolved["team_id"] = team_id
#         elif role == "against":
#             resolved["opponent_team_id"] = team_id

#     if entities.get("league"):
#         league_name = entities["league"]
#         resolved["league_id"] = 39 if league_name and league_name.lower() == "premier league" else get_league_id(league_name)

#     if entities.get("season"):
#         try:
#             resolved["season_year"] = int(entities["season"])
#         except:
#             resolved["season_year"] = None

#     if isinstance(entities.get("event_type"), dict):
#         etype = entities["event_type"].get("type")
#         edetail = entities["event_type"].get("detail")
#         normalized = normalize_event_type_pair(etype, edetail)
#         if normalized in VALID_EVENT_TYPES:
#             resolved["event_type"] = normalized[0]
#             resolved["event_detail"] = normalized[1]
#         else:
#             print("\u26a0\ufe0f Invalid event type/detail pair:", normalized)

#     resolved["raw_entities"] = entities
#     return resolved

# # === 9. QUERY GENERATOR ===
# def build_filtered_event_query(filters: dict) -> str:
#     base_query = """
#     SELECT
#       e.event_id,
#       e.type,
#       p.name AS player_name,
#       e.detail,
#       e.comments,
#       e.minute,
#       e.extra_minute,
#       f.fixture_id,
#       f.referee,
#       f.fixture_date,
#       f.venue_id,
#       f.season_year,
#       f.league_id,
#       f.league_round,
#       f.total_home_goals,
#       f.total_away_goals,
#       e.team_id AS player_team_id,
#       t.name AS player_team_name,
#       CASE
#         WHEN e.team_id = f.home_team_id THEN f.away_team_id
#         ELSE f.home_team_id
#       END AS opponent_team_id,
#       CASE
#         WHEN e.team_id = f.home_team_id THEN (SELECT name FROM base_data_apis.teams WHERE team_id = f.away_team_id)
#         ELSE (SELECT name FROM base_data_apis.teams WHERE team_id = f.home_team_id)
#       END AS opponent_team_name,
#       a.name AS assist_name
#     FROM base_data_apis.fixture_events e
#     JOIN base_data_apis.fixtures f ON e.fixture_id = f.fixture_id
#       AND (e.team_id = f.home_team_id OR e.team_id = f.away_team_id)
#     LEFT JOIN base_data_apis.players p ON e.player_id = p.player_id
#     LEFT JOIN base_data_apis.players a ON e.assist_id = a.player_id
#     LEFT JOIN base_data_apis.teams t ON e.team_id = t.team_id
#     WHERE 1=1
#     """

#     conditions = []

#     if filters.get("player_id"):
#         conditions.append(f"AND e.player_id = {filters['player_id']}")
#     if filters.get("team_id"):
#         conditions.append(f"AND e.team_id = {filters['team_id']}")
#     if filters.get("opponent_team_id"):
#         conditions.append(f"AND (f.home_team_id = {filters['opponent_team_id']} OR f.away_team_id = {filters['opponent_team_id']})")
#         conditions.append(f"AND e.team_id != {filters['opponent_team_id']}")
#     if filters.get("league_id"):
#         conditions.append(f"AND f.league_id = {filters['league_id']}")
#     if filters.get("season_year"):
#         conditions.append(f"AND f.season_year = {filters['season_year']}")
#     if filters.get("event_type"):
#         conditions.append(f"AND e.type = '{filters['event_type']}'")
#     if filters.get("event_detail"):
#         conditions.append(f"AND e.detail = '{filters['event_detail']}'")

#     return base_query + "\n".join(conditions) + ";"

# # === 10. EXECUTOR ===
# def run_sql_query(query: str) -> str:
#     try:
#         conn = mysql.connector.connect(
#             host=DB_HOST, user=DB_USER, password=DB_PASS, database=DB_NAME, port=DB_PORT
#         )
#         cursor = conn.cursor()
#         cursor.execute(query)
#         rows = cursor.fetchall()
#         columns = [desc[0] for desc in cursor.description]
#         conn.close()

#         if not rows:
#             return "No results found."

#         return "Columns: " + str(columns) + "\n" + "\n".join([str(row) for row in rows])
#     except Exception as e:
#         return f"❌ Error running query: {str(e)}"

# # === 11. DEMO RUN ===
# if __name__ == "__main__":
#     user_query = "How many goals did Erling Haaland score against Manchester United in Premier League in 2023?"
#     filters = resolve_entities_to_ids(user_query)
#     sql = build_filtered_event_query(filters)
#     print("\n📄 Final SQL Query:")
#     print(sql)
#     result = run_sql_query(sql)
#     print("\n📊 Query Result:")
#     print(result)




> Entering new LLMChain chain...
Prompt after formatting:

You are a football data assistant. Extract and classify entities from the user's query into this JSON:

{
  "player": <player_name or null>,
  "team": <team_name or null>,
  "team_role": "for" | "against" | null,
  "league": <league_name or null>,
  "season": <season_year or null>,
  "event_type": {
    "type": <Goal|Card|subst|Var or null>,
    "detail": <valid detail string or null>
  },
  "referee": <referee_name or null>,
  "round": <round number or null>,
  "venue": <venue name or null>
}

Valid event type + detail combinations include:
Goal → Penalty, Normal Goal, Own Goal, Missed Penalty
Card → Yellow Card, Red Card, (empty)
subst → Substitution (covers all numbered substitutions)
Var → Goal cancelled, Penalty cancelled, Card upgrade, Goal confirmed, Card reviewed, Penalty awarded, Penalty confirmed, Red card cancelled, Goal Disallowed, Goal Disallowed - offside, Goal Disallowed - handball, Goal Disallowed - Foul, (emp

In [ ]:
# # ✅ END-TO-END AGENT FOR ENTITY EXTRACTION + ID RESOLUTION + DYNAMIC SQL QUERY EXECUTION

# from langchain.chains import LLMChain
# from langchain.prompts import PromptTemplate
# from langchain_nvidia_ai_endpoints import ChatNVIDIA
# import mysql.connector
# import os
# import json
# from rapidfuzz import process, fuzz

# # === 1. SETUP NVIDIA LLM ===
# llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1", api_key=NVIDIA_API_KEY)

# # === 2. ENTITY EXTRACTION PROMPT ===
# entity_extraction_prompt = PromptTemplate(
#     input_variables=["query"],
#     template="""
# You are a football data assistant. Extract and classify entities from the user's query into this JSON:

# {{
#   "player": <player_name or null>,
#   "team": <team_name or null>,
#   "team_role": "for" | "against" | null,
#   "league": <league_name or null>,
#   "season": <season_year or null>,
#   "event_type": {{
#     "type": <Goal|Card|subst|Var or null>,
#     "detail": <valid detail string or null>
#   }},
#   "referee": <referee_name or null>,
#   "round": <round number or null>,
#   "venue": <venue name or null>
# }}

# Valid event type + detail combinations include:
# Goal → Penalty, Normal Goal, Own Goal, Missed Penalty
# Card → Yellow Card, Red Card, (empty)
# subst → Substitution (covers all numbered substitutions)
# Var → Goal cancelled, Penalty cancelled, Card upgrade, Goal confirmed, Card reviewed, Penalty awarded, Penalty confirmed, Red card cancelled, Goal Disallowed, Goal Disallowed - offside, Goal Disallowed - handball, Goal Disallowed - Foul, (empty)

# Always normalize substitution details to just "Substitution"

# User query: "{query}"
# """
# )

# # === 3. ENTITY EXTRACTION CHAIN ===
# entity_chain = LLMChain(llm=llm, prompt=entity_extraction_prompt, verbose=True)

# def extract_entities(user_query: str) -> dict:
#     try:
#         response = entity_chain.run(query=user_query)
#         return json.loads(response)
#     except Exception as e:
#         print("\u26a0\ufe0f Failed to parse entity JSON:", str(e))
#         return {}

# # === 5. FUZZY MATCHING LOOKUP ===
# def fuzzy_lookup(query_name: str, table: str, column: str, id_column: str, threshold: int = 75) -> int:
#     try:
#         conn = mysql.connector.connect(
#             host=DB_HOST, user=DB_USER, password=DB_PASS, database=DB_NAME, port=DB_PORT
#         )
#         cursor = conn.cursor()
#         cursor.execute(f"SELECT {id_column}, {column} FROM {table}")
#         rows = cursor.fetchall()
#         conn.close()

#         name_dict = {row[1]: row[0] for row in rows if row[1]}
#         best_match = process.extractOne(query_name, name_dict.keys(), scorer=fuzz.token_sort_ratio)

#         if best_match and best_match[1] >= threshold:
#             print(f"✅ Fuzzy matched '{query_name}' → '{best_match[0]}' ({best_match[1]}%)")
#             return name_dict[best_match[0]]
#         else:
#             print(f"⚠️ No good fuzzy match for '{query_name}' (best: {best_match})")
#             return None
#     except Exception as e:
#         print(f"❌ Error in fuzzy lookup for '{query_name}': {str(e)}")
#         return None

# # === 6. ENTITY -> ID FUNCTIONS ===
# def get_player_id(name: str) -> int:
#     return fuzzy_lookup(name, "base_data_apis.players", "name", "player_id")

# def get_team_id(name: str) -> int:
#     return fuzzy_lookup(name, "base_data_apis.teams", "name", "team_id")

# def get_league_id(name: str) -> int:
#     return fuzzy_lookup(name, "base_data_apis.leagues", "name", "league_id")

# # === 7. VALID EVENT TYPES ===
# VALID_EVENT_TYPES = {
#     ("Goal", "Penalty"), ("Goal", "Normal Goal"), ("Goal", "Own Goal"), ("Goal", "Missed Penalty"),
#     ("Card", "Yellow Card"), ("Card", "Red Card"), ("Card", ""),
#     ("Var", "Goal cancelled"), ("Var", "Penalty cancelled"), ("Var", "Card upgrade"), ("Var", "Goal confirmed"),
#     ("Var", "Card reviewed"), ("Var", "Penalty awarded"), ("Var", "Penalty confirmed"), ("Var", "Red card cancelled"),
#     ("Var", "Goal Disallowed"), ("Var", "Goal Disallowed - offside"), ("Var", "Goal Disallowed - handball"),
#     ("Var", "Goal Disallowed - Foul"), ("Var", ""),
#     ("subst", "Substitution")
# }

# def normalize_event_type_pair(event_type: str, event_detail: str) -> tuple:
#     if event_type == "subst" and event_detail and event_detail.lower().startswith("substitution"):
#         return ("subst", "Substitution")
#     return (event_type or "", event_detail or "")

# # === 8. MAIN AGENT FUNCTION ===
# def resolve_entities_to_ids(user_query: str) -> dict:
#     entities = extract_entities(user_query)
#     resolved = {}

#     if entities.get("player"):
#         resolved["player_id"] = get_player_id(entities["player"])

#     if entities.get("team"):
#         team_id = get_team_id(entities["team"])
#         role = entities.get("team_role", "for")
#         if role == "for":
#             resolved["team_id"] = team_id
#         elif role == "against":
#             resolved["opponent_team_id"] = team_id

#     if entities.get("league"):
#         league_name = entities["league"]
#         resolved["league_id"] = 39 if league_name and league_name.lower() == "premier league" else get_league_id(league_name)

#     if entities.get("season"):
#         try:
#             resolved["season_year"] = int(entities["season"])
#         except:
#             resolved["season_year"] = None

#     if isinstance(entities.get("event_type"), dict):
#         etype = entities["event_type"].get("type")
#         edetail = entities["event_type"].get("detail")

#         if etype in {"Goal", "Card", "Var", "subst"} and (edetail is None or edetail.strip() == ""):
#             resolved["event_type"] = etype
#         else:
#             normalized = normalize_event_type_pair(etype, edetail)
#             if normalized in VALID_EVENT_TYPES:
#                 resolved["event_type"] = normalized[0]
#                 resolved["event_detail"] = normalized[1]
#             else:
#                 print("⚠️ Invalid event type/detail pair:", normalized)

#     resolved["raw_entities"] = entities
#     return resolved

# # === 9. QUERY GENERATOR ===
# def build_filtered_event_query(filters: dict) -> str:
#     base_query = """
#     SELECT
#       e.event_id,
#       e.type,
#       p.name AS player_name,
#       e.detail,
#       e.comments,
#       e.minute,
#       e.extra_minute,
#       f.fixture_id,
#       f.referee,
#       f.fixture_date,
#       f.venue_id,
#       f.season_year,
#       f.league_id,
#       f.league_round,
#       f.total_home_goals,
#       f.total_away_goals,
#       e.team_id AS player_team_id,
#       t.name AS player_team_name,
#       CASE
#         WHEN e.team_id = f.home_team_id THEN f.away_team_id
#         ELSE f.home_team_id
#       END AS opponent_team_id,
#       CASE
#         WHEN e.team_id = f.home_team_id THEN (SELECT name FROM base_data_apis.teams WHERE team_id = f.away_team_id)
#         ELSE (SELECT name FROM base_data_apis.teams WHERE team_id = f.home_team_id)
#       END AS opponent_team_name,
#       a.name AS assist_name
#     FROM base_data_apis.fixture_events e
#     JOIN base_data_apis.fixtures f ON e.fixture_id = f.fixture_id
#       AND (e.team_id = f.home_team_id OR e.team_id = f.away_team_id)
#     LEFT JOIN base_data_apis.players p ON e.player_id = p.player_id
#     LEFT JOIN base_data_apis.players a ON e.assist_id = a.player_id
#     LEFT JOIN base_data_apis.teams t ON e.team_id = t.team_id
#     WHERE 1=1
#     """

#     conditions = []

#     if filters.get("player_id"):
#         conditions.append(f"AND e.player_id = {filters['player_id']}")
#     if filters.get("team_id"):
#         conditions.append(f"AND e.team_id = {filters['team_id']}")
#     if filters.get("opponent_team_id"):
#         conditions.append(f"AND (f.home_team_id = {filters['opponent_team_id']} OR f.away_team_id = {filters['opponent_team_id']})")
#         conditions.append(f"AND e.team_id != {filters['opponent_team_id']}")
#     if filters.get("league_id"):
#         conditions.append(f"AND f.league_id = {filters['league_id']}")
#     if filters.get("season_year"):
#         conditions.append(f"AND f.season_year = {filters['season_year']}")
#     if filters.get("event_type"):
#         conditions.append(f"AND e.type = '{filters['event_type']}'")
#     if filters.get("event_detail") and filters["event_detail"].strip():
#         conditions.append(f"AND e.detail = '{filters['event_detail']}'")

#     return base_query + "\n".join(conditions) + ";"

# # === 10. EXECUTOR ===
# def run_sql_query(query: str) -> str:
#     try:
#         conn = mysql.connector.connect(
#             host=DB_HOST, user=DB_USER, password=DB_PASS, database=DB_NAME, port=DB_PORT
#         )
#         cursor = conn.cursor()
#         cursor.execute(query)
#         rows = cursor.fetchall()
#         columns = [desc[0] for desc in cursor.description]
#         conn.close()

#         if not rows:
#             return "No results found."

#         return "Columns: " + str(columns) + "\n" + "\n".join([str(row) for row in rows])
#     except Exception as e:
#         return f"❌ Error running query: {str(e)}"

# # === 11. DEMO RUN ===
# if __name__ == "__main__":
#     user_query = "How many goals did Erling Haaland score against Manchester United in Premier League in 2023?"
#     filters = resolve_entities_to_ids(user_query)
#     sql = build_filtered_event_query(filters)
#     print("\n📄 Final SQL Query:")
#     print(sql)
#     result = run_sql_query(sql)
#     print("\n📊 Query Result:")
#     print(result)




> Entering new LLMChain chain...
Prompt after formatting:

You are a football data assistant. Extract and classify entities from the user's query into this JSON:

{
  "player": <player_name or null>,
  "team": <team_name or null>,
  "team_role": "for" | "against" | null,
  "league": <league_name or null>,
  "season": <season_year or null>,
  "event_type": {
    "type": <Goal|Card|subst|Var or null>,
    "detail": <valid detail string or null>
  },
  "referee": <referee_name or null>,
  "round": <round number or null>,
  "venue": <venue name or null>
}

Valid event type + detail combinations include:
Goal → Penalty, Normal Goal, Own Goal, Missed Penalty
Card → Yellow Card, Red Card, (empty)
subst → Substitution (covers all numbered substitutions)
Var → Goal cancelled, Penalty cancelled, Card upgrade, Goal confirmed, Card reviewed, Penalty awarded, Penalty confirmed, Red card cancelled, Goal Disallowed, Goal Disallowed - offside, Goal Disallowed - handball, Goal Disallowed - Foul, (emp

In [ ]:
# # ✅ END-TO-END AGENT FOR ENTITY EXTRACTION + ID RESOLUTION + DYNAMIC SQL QUERY EXECUTION

# from langchain.chains import LLMChain
# from langchain.prompts import PromptTemplate
# from langchain_nvidia_ai_endpoints import ChatNVIDIA
# import mysql.connector
# import os
# import json
# from rapidfuzz import process, fuzz

# # === 1. SETUP NVIDIA LLM ===
# llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1", api_key=NVIDIA_API_KEY)

# # === 2. ENTITY EXTRACTION PROMPT ===
# entity_extraction_prompt = PromptTemplate(
#     input_variables=["query"],
#     template="""
# You are a football data assistant. Extract and classify entities from the user's query into this JSON:

# {{
#   "player": <player_name or null>,
#   "team": <full team name (e.g., Manchester City) or null>,
#   "team_role": "for" | "against" | null,
#   "league": <league_name or null>,
#   "season": <season_year or null>,
#   "event_type": {{
#     "type": <Goal|Card|subst|Var or null>,
#     "detail": <valid detail string or null>
#   }},
#   "referee": <referee_name or null>,
#   "round": <round number or null>,
#   "venue": <venue name or null>
# }}

# Valid event type + detail combinations include:
# Goal → Penalty, Normal Goal, Own Goal, Missed Penalty
# Card → Yellow Card, Red Card, (empty)
# subst → Substitution (covers all numbered substitutions)
# Var → Goal cancelled, Penalty cancelled, Card upgrade, Goal confirmed, Card reviewed, Penalty awarded, Penalty confirmed, Red card cancelled, Goal Disallowed, Goal Disallowed - offside, Goal Disallowed - handball, Goal Disallowed - Foul, (empty)

# Always normalize substitution details to just "Substitution"

# User query: "{query}"
# """
# )

# # === 3. ENTITY EXTRACTION CHAIN ===
# entity_chain = LLMChain(llm=llm, prompt=entity_extraction_prompt, verbose=True)

# def extract_entities(user_query: str) -> dict:
#     try:
#         response = entity_chain.run(query=user_query)
#         return json.loads(response)
#     except Exception as e:
#         print("\u26a0\ufe0f Failed to parse entity JSON:", str(e))
#         return {}

# # === 5. FUZZY MATCHING LOOKUP ===
# def fuzzy_lookup(query_name: str, table: str, column: str, id_column: str, threshold: int = 75) -> int:
#     try:
#         conn = mysql.connector.connect(
#             host=DB_HOST, user=DB_USER, password=DB_PASS, database=DB_NAME, port=DB_PORT
#         )
#         cursor = conn.cursor()
#         cursor.execute(f"SELECT {id_column}, {column} FROM {table}")
#         rows = cursor.fetchall()
#         conn.close()

#         name_dict = {row[1]: row[0] for row in rows if row[1]}
#         filtered_matches = process.extract(
#             query_name, name_dict.keys(), scorer=fuzz.token_sort_ratio, limit=5
#         )
#         filtered_matches = [m for m in filtered_matches if m[1] >= threshold]
#         if filtered_matches:
#             best_match = sorted(filtered_matches, key=lambda x: (x[1], len(x[0])), reverse=True)[0]
#             print(f"✅ Fuzzy matched '{query_name}' → '{best_match[0]}' ({best_match[1]}%)")
#             return name_dict[best_match[0]]
#         else:
#             print(f"⚠️ No good fuzzy match for '{query_name}'")
#             return None
#     except Exception as e:
#         print(f"❌ Error in fuzzy lookup for '{query_name}': {str(e)}")
#         return None

# # === 6. ENTITY -> ID FUNCTIONS ===
# def get_player_id(name: str) -> int:
#     return fuzzy_lookup(name, "base_data_apis.players", "name", "player_id")

# def get_team_id(name: str) -> int:
#     return fuzzy_lookup(name, "base_data_apis.teams", "name", "team_id")

# def get_league_id(name: str) -> int:
#     return fuzzy_lookup(name, "base_data_apis.leagues", "name", "league_id")

# # === 7. VALID EVENT TYPES ===
# VALID_EVENT_TYPES = {
#     ("Goal", "Penalty"), ("Goal", "Normal Goal"), ("Goal", "Own Goal"), ("Goal", "Missed Penalty"),
#     ("Card", "Yellow Card"), ("Card", "Red Card"), ("Card", ""),
#     ("Var", "Goal cancelled"), ("Var", "Penalty cancelled"), ("Var", "Card upgrade"), ("Var", "Goal confirmed"),
#     ("Var", "Card reviewed"), ("Var", "Penalty awarded"), ("Var", "Penalty confirmed"), ("Var", "Red card cancelled"),
#     ("Var", "Goal Disallowed"), ("Var", "Goal Disallowed - offside"), ("Var", "Goal Disallowed - handball"),
#     ("Var", "Goal Disallowed - Foul"), ("Var", ""),
#     ("subst", "Substitution")
# }

# def normalize_event_type_pair(event_type: str, event_detail: str) -> tuple:
#     if event_type == "subst" and event_detail and event_detail.lower().startswith("substitution"):
#         return ("subst", "Substitution")
#     return (event_type or "", event_detail or "")

# # === 8. MAIN AGENT FUNCTION ===
# def resolve_entities_to_ids(user_query: str) -> dict:
#     entities = extract_entities(user_query)
#     resolved = {}

#     if entities.get("player"):
#         resolved["player_id"] = get_player_id(entities["player"])

#     if entities.get("team"):
#         team_id = get_team_id(entities["team"])
#         role = entities.get("team_role", "for")
#         if role == "for":
#             resolved["team_id"] = team_id
#         elif role == "against":
#             resolved["opponent_team_id"] = team_id

#     if entities.get("league"):
#         league_name = entities["league"]
#         resolved["league_id"] = 39 if league_name and league_name.lower() == "premier league" else get_league_id(league_name)

#     if entities.get("season"):
#         try:
#             resolved["season_year"] = int(entities["season"])
#         except:
#             resolved["season_year"] = None

#     if isinstance(entities.get("event_type"), dict):
#         etype = entities["event_type"].get("type")
#         edetail = entities["event_type"].get("detail")

#         if etype in {"Goal", "Card", "Var", "subst"} and (edetail is None or edetail.strip() == ""):
#             resolved["event_type"] = etype
#         else:
#             normalized = normalize_event_type_pair(etype, edetail)
#             if normalized in VALID_EVENT_TYPES:
#                 resolved["event_type"] = normalized[0]
#                 resolved["event_detail"] = normalized[1]
#             else:
#                 print("⚠️ Invalid event type/detail pair:", normalized)

#     resolved["raw_entities"] = entities
#     return resolved

# # === 9. QUERY GENERATOR ===
# def build_filtered_event_query(filters: dict) -> str:
#     base_query = """
#     SELECT
#       e.event_id,
#       e.type,
#       p.name AS player_name,
#       e.detail,
#       e.comments,
#       e.minute,
#       e.extra_minute,
#       f.fixture_id,
#       f.referee,
#       f.fixture_date,
#       f.venue_id,
#       f.season_year,
#       f.league_id,
#       f.league_round,
#       f.total_home_goals,
#       f.total_away_goals,
#       e.team_id AS player_team_id,
#       t.name AS player_team_name,
#       CASE
#         WHEN e.team_id = f.home_team_id THEN f.away_team_id
#         ELSE f.home_team_id
#       END AS opponent_team_id,
#       CASE
#         WHEN e.team_id = f.home_team_id THEN (SELECT name FROM base_data_apis.teams WHERE team_id = f.away_team_id)
#         ELSE (SELECT name FROM base_data_apis.teams WHERE team_id = f.home_team_id)
#       END AS opponent_team_name,
#       a.name AS assist_name
#     FROM base_data_apis.fixture_events e
#     JOIN base_data_apis.fixtures f ON e.fixture_id = f.fixture_id
#       AND (e.team_id = f.home_team_id OR e.team_id = f.away_team_id)
#     LEFT JOIN base_data_apis.players p ON e.player_id = p.player_id
#     LEFT JOIN base_data_apis.players a ON e.assist_id = a.player_id
#     LEFT JOIN base_data_apis.teams t ON e.team_id = t.team_id
#     WHERE 1=1
#     """

#     conditions = []

#     if filters.get("player_id"):
#         conditions.append(f"AND e.player_id = {filters['player_id']}")
#     if filters.get("team_id"):
#         conditions.append(f"AND e.team_id = {filters['team_id']}")
#     if filters.get("opponent_team_id"):
#         conditions.append(f"AND (f.home_team_id = {filters['opponent_team_id']} OR f.away_team_id = {filters['opponent_team_id']})")
#         conditions.append(f"AND e.team_id != {filters['opponent_team_id']}")
#     if filters.get("league_id"):
#         conditions.append(f"AND f.league_id = {filters['league_id']}")
#     if filters.get("season_year"):
#         conditions.append(f"AND f.season_year = {filters['season_year']}")
#     if filters.get("event_type"):
#         conditions.append(f"AND e.type = '{filters['event_type']}'")
#     if filters.get("event_detail") and filters["event_detail"].strip():
#         conditions.append(f"AND e.detail = '{filters['event_detail']}'")

#     return base_query + "\n".join(conditions) + ";"

# # === 10. EXECUTOR ===
# def run_sql_query(query: str) -> str:
#     try:
#         conn = mysql.connector.connect(
#             host=DB_HOST, user=DB_USER, password=DB_PASS, database=DB_NAME, port=DB_PORT
#         )
#         cursor = conn.cursor()
#         cursor.execute(query)
#         rows = cursor.fetchall()
#         columns = [desc[0] for desc in cursor.description]
#         conn.close()

#         if not rows:
#             return "No results found."

#         return "Columns: " + str(columns) + "\n" + "\n".join([str(row) for row in rows])
#     except Exception as e:
#         return f"❌ Error running query: {str(e)}"

# # === 11. DEMO RUN ===
# if __name__ == "__main__":
#     user_query = "How many goals did Rashford score for United in Premier League in 2023?"
#     filters = resolve_entities_to_ids(user_query)
#     sql = build_filtered_event_query(filters)
#     print("\n📄 Final SQL Query:")
#     print(sql)
#     result = run_sql_query(sql)
#     print("\n📊 Query Result:")
#     print(result)




> Entering new LLMChain chain...
Prompt after formatting:

You are a football data assistant. Extract and classify entities from the user's query into this JSON:

{
  "player": <player_name or null>,
  "team": <full team name (e.g., Manchester City) or null>,
  "team_role": "for" | "against" | null,
  "league": <league_name or null>,
  "season": <season_year or null>,
  "event_type": {
    "type": <Goal|Card|subst|Var or null>,
    "detail": <valid detail string or null>
  },
  "referee": <referee_name or null>,
  "round": <round number or null>,
  "venue": <venue name or null>
}

Valid event type + detail combinations include:
Goal → Penalty, Normal Goal, Own Goal, Missed Penalty
Card → Yellow Card, Red Card, (empty)
subst → Substitution (covers all numbered substitutions)
Var → Goal cancelled, Penalty cancelled, Card upgrade, Goal confirmed, Card reviewed, Penalty awarded, Penalty confirmed, Red card cancelled, Goal Disallowed, Goal Disallowed - offside, Goal Disallowed - handball,

In [ ]:
# # ✅ END-TO-END AGENT FOR ENTITY EXTRACTION + ID RESOLUTION + DYNAMIC SQL QUERY EXECUTION

# from langchain.chains import LLMChain
# from langchain.prompts import PromptTemplate
# from langchain_nvidia_ai_endpoints import ChatNVIDIA
# import mysql.connector
# import os
# import json
# from rapidfuzz import process, fuzz

# # === 1. SETUP NVIDIA LLM ===
# llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1", api_key=NVIDIA_API_KEY)

# # === 2. ENTITY EXTRACTION PROMPT ===
# entity_extraction_prompt = PromptTemplate(
#     input_variables=["query"],
#     template="""
# You are a football data assistant. Extract and classify entities from the user's query into this JSON:

# {{
#   "player": <player_name or null>,
#   "team": <full team name (e.g., Manchester City) or null>,
#   "team_role": "for" | "against" | null,
#   "league": <league_name or null>,
#   "season": <season_year or null>,
#   "event_type": {{
#     "type": <Goal|Card|subst|Var or null>,
#     "detail": <valid detail string or null>
#   }},
#   "referee": <referee_name or null>,
#   "round": <round number or null>,
#   "venue": <venue name or null>
# }}

# Valid event type + detail combinations include:
# Goal → Penalty, Normal Goal, Own Goal, Missed Penalty
# Card → Yellow Card, Red Card, (empty)
# subst → Substitution (covers all numbered substitutions)
# Var → Goal cancelled, Penalty cancelled, Card upgrade, Goal confirmed, Card reviewed, Penalty awarded, Penalty confirmed, Red card cancelled, Goal Disallowed, Goal Disallowed - offside, Goal Disallowed - handball, Goal Disallowed - Foul, (empty)

# Always normalize substitution details to just "Substitution"

# User query: "{query}"
# """
# )

# # === 3. ENTITY EXTRACTION CHAIN ===
# entity_chain = LLMChain(llm=llm, prompt=entity_extraction_prompt, verbose=True)

# def extract_entities(user_query: str) -> dict:
#     try:
#         response = entity_chain.run(query=user_query)
#         return json.loads(response)
#     except Exception as e:
#         print("\u26a0\ufe0f Failed to parse entity JSON:", str(e))
#         return {}

# # === 5. FUZZY MATCHING LOOKUP ===
# def fuzzy_lookup(query_name: str, table: str, column: str, id_column: str, threshold: int = 75) -> int:
#     try:
#         conn = mysql.connector.connect(
#             host=DB_HOST, user=DB_USER, password=DB_PASS, database=DB_NAME, port=DB_PORT
#         )
#         cursor = conn.cursor()
#         cursor.execute(f"SELECT {id_column}, {column} FROM {table}")
#         rows = cursor.fetchall()
#         conn.close()

#         name_dict = {row[1]: row[0] for row in rows if row[1]}
#         filtered_matches = process.extract(
#             query_name, name_dict.keys(), scorer=fuzz.token_sort_ratio, limit=5
#         )
#         filtered_matches = [m for m in filtered_matches if m[1] >= threshold]
#         if filtered_matches:
#             best_match = sorted(filtered_matches, key=lambda x: (x[1], len(x[0])), reverse=True)[0]
#             print(f"✅ Fuzzy matched '{query_name}' → '{best_match[0]}' ({best_match[1]}%)")
#             return name_dict[best_match[0]]
#         else:
#             print(f"⚠️ No good fuzzy match for '{query_name}'")
#             return None
#     except Exception as e:
#         print(f"❌ Error in fuzzy lookup for '{query_name}': {str(e)}")
#         return None

# # === 6. ENTITY -> ID FUNCTIONS ===
# def get_player_id(name: str) -> int:
#     return fuzzy_lookup(name, "base_data_apis.players", "name", "player_id")

# def get_team_id(name: str) -> int:
#     return fuzzy_lookup(name, "base_data_apis.teams", "name", "team_id")

# def get_league_id(name: str) -> int:
#     return fuzzy_lookup(name, "base_data_apis.leagues", "name", "league_id")

# # === 7. VALID EVENT TYPES ===
# VALID_EVENT_TYPES = {
#     ("Goal", "Penalty"), ("Goal", "Normal Goal"), ("Goal", "Own Goal"), ("Goal", "Missed Penalty"),
#     ("Card", "Yellow Card"), ("Card", "Red Card"), ("Card", ""),
#     ("Var", "Goal cancelled"), ("Var", "Penalty cancelled"), ("Var", "Card upgrade"), ("Var", "Goal confirmed"),
#     ("Var", "Card reviewed"), ("Var", "Penalty awarded"), ("Var", "Penalty confirmed"), ("Var", "Red card cancelled"),
#     ("Var", "Goal Disallowed"), ("Var", "Goal Disallowed - offside"), ("Var", "Goal Disallowed - handball"),
#     ("Var", "Goal Disallowed - Foul"), ("Var", ""),
#     ("subst", "Substitution")
# }

# def normalize_event_type_pair(event_type: str, event_detail: str) -> tuple:
#     if event_type == "subst" and event_detail and event_detail.lower().startswith("substitution"):
#         return ("subst", "Substitution")
#     return (event_type or "", event_detail or "")

# # === 8. MAIN AGENT FUNCTION ===
# def resolve_entities_to_ids(user_query: str) -> dict:
#     entities = extract_entities(user_query)
#     resolved = {}

#     if entities.get("player"):
#         resolved["player_id"] = get_player_id(entities["player"])

#     if entities.get("team"):
#         team_id = get_team_id(entities["team"])
#         role = entities.get("team_role", "for")
#         if role == "for":
#             resolved["team_id"] = team_id
#         elif role == "against":
#             resolved["opponent_team_id"] = team_id

#     if entities.get("league"):
#         league_name = entities["league"]
#         resolved["league_id"] = 39 if league_name and league_name.lower() == "premier league" else get_league_id(league_name)

#     if entities.get("season"):
#         try:
#             resolved["season_year"] = int(entities["season"])
#         except:
#             resolved["season_year"] = None

#     if isinstance(entities.get("event_type"), dict):
#         etype = entities["event_type"].get("type")
#         edetail = entities["event_type"].get("detail")

#         if etype in {"Goal", "Card", "Var", "subst"} and (edetail is None or edetail.strip() == ""):
#             resolved["event_type"] = etype
#         else:
#             normalized = normalize_event_type_pair(etype, edetail)
#             if normalized in VALID_EVENT_TYPES:
#                 resolved["event_type"] = normalized[0]
#                 resolved["event_detail"] = normalized[1]
#             else:
#                 print("⚠️ Invalid event type/detail pair:", normalized)

#     resolved["raw_entities"] = entities
#     return resolved

# # === 9. QUERY GENERATOR ===
# def build_filtered_event_query(filters: dict) -> str:
#     base_query = """
#     SELECT
#       e.event_id,
#       e.type,
#       p.name AS player_name,
#       e.detail,
#       e.comments,
#       e.minute,
#       e.extra_minute,
#       f.fixture_id,
#       f.referee,
#       f.fixture_date,
#       f.venue_id,
#       f.season_year,
#       f.league_id,
#       f.league_round,
#       f.total_home_goals,
#       f.total_away_goals,
#       e.team_id AS player_team_id,
#       t.name AS player_team_name,
#       CASE
#         WHEN e.team_id = f.home_team_id THEN f.away_team_id
#         ELSE f.home_team_id
#       END AS opponent_team_id,
#       CASE
#         WHEN e.team_id = f.home_team_id THEN (SELECT name FROM base_data_apis.teams WHERE team_id = f.away_team_id)
#         ELSE (SELECT name FROM base_data_apis.teams WHERE team_id = f.home_team_id)
#       END AS opponent_team_name,
#       a.name AS assist_name
#     FROM base_data_apis.fixture_events e
#     JOIN base_data_apis.fixtures f ON e.fixture_id = f.fixture_id
#       AND (e.team_id = f.home_team_id OR e.team_id = f.away_team_id)
#     LEFT JOIN base_data_apis.players p ON e.player_id = p.player_id
#     LEFT JOIN base_data_apis.players a ON e.assist_id = a.player_id
#     LEFT JOIN base_data_apis.teams t ON e.team_id = t.team_id
#     WHERE 1=1
#     """

#     conditions = []

#     if filters.get("player_id"):
#         conditions.append(f"AND e.player_id = {filters['player_id']}")
#     if filters.get("team_id"):
#         conditions.append(f"AND e.team_id = {filters['team_id']}")
#     if filters.get("opponent_team_id"):
#         conditions.append(f"AND (f.home_team_id = {filters['opponent_team_id']} OR f.away_team_id = {filters['opponent_team_id']})")
#         conditions.append(f"AND e.team_id != {filters['opponent_team_id']}")
#     if filters.get("league_id"):
#         conditions.append(f"AND f.league_id = {filters['league_id']}")
#     if filters.get("season_year"):
#         conditions.append(f"AND f.season_year = {filters['season_year']}")
#     if filters.get("event_type"):
#         conditions.append(f"AND e.type = '{filters['event_type']}'")
#     if filters.get("event_detail") and filters["event_detail"].strip():
#         conditions.append(f"AND e.detail = '{filters['event_detail']}'")

#     return base_query + "\n".join(conditions) + ";"

# # === 10. EXECUTOR ===
# def run_sql_query(query: str) -> str:
#     try:
#         conn = mysql.connector.connect(
#             host=DB_HOST, user=DB_USER, password=DB_PASS, database=DB_NAME, port=DB_PORT
#         )
#         cursor = conn.cursor()
#         cursor.execute(query)
#         rows = cursor.fetchall()
#         columns = [desc[0] for desc in cursor.description]
#         conn.close()

#         if not rows:
#             return "No results found."

#         return "Columns: " + str(columns) + "\n" + "\n".join([str(row) for row in rows])
#     except Exception as e:
#         return f"❌ Error running query: {str(e)}"

# # === 11. DEMO RUN ===
# if __name__ == "__main__":
#     user_query = "How many goals did Rashford score for United in Premier League in 2023?"
#     filters = resolve_entities_to_ids(user_query)
#     sql = build_filtered_event_query(filters)
#     print("\n📄 Final SQL Query:")
#     print(sql)
#     result = run_sql_query(sql)
#     print("\n📊 Query Result:")
#     print(result)




> Entering new LLMChain chain...
Prompt after formatting:

You are a football data assistant. Extract and classify entities from the user's query into this JSON:

{
  "player": <player_name or null>,
  "team": <full team name (e.g., Manchester City) or null>,
  "team_role": "for" | "against" | null,
  "league": <league_name or null>,
  "season": <season_year or null>,
  "event_type": {
    "type": <Goal|Card|subst|Var or null>,
    "detail": <valid detail string or null>
  },
  "referee": <referee_name or null>,
  "round": <round number or null>,
  "venue": <venue name or null>
}

Valid event type + detail combinations include:
Goal → Penalty, Normal Goal, Own Goal, Missed Penalty
Card → Yellow Card, Red Card, (empty)
subst → Substitution (covers all numbered substitutions)
Var → Goal cancelled, Penalty cancelled, Card upgrade, Goal confirmed, Card reviewed, Penalty awarded, Penalty confirmed, Red card cancelled, Goal Disallowed, Goal Disallowed - offside, Goal Disallowed - handball,

In [ ]:
# # ✅ END-TO-END AGENT FOR ENTITY EXTRACTION + ID RESOLUTION + DYNAMIC SQL QUERY EXECUTION + AGGREGATION

# from langchain.chains import LLMChain
# from langchain.prompts import PromptTemplate
# from langchain_nvidia_ai_endpoints import ChatNVIDIA
# import mysql.connector
# import os
# import json
# from rapidfuzz import process, fuzz

# # === 1. SETUP NVIDIA LLM ===
# llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1", api_key=NVIDIA_API_KEY)

# # === 2. ENTITY EXTRACTION PROMPT ===
# entity_extraction_prompt = PromptTemplate(
#     input_variables=["query"],
#     template="""
# You are a football data assistant. Extract and classify entities from the user's query into this JSON:

# {{
#   "player": <player_name or null>,
#   "team": <full team name (e.g., Manchester City) or null>,
#   "team_role": "for" | "against" | null,
#   "league": <league_name or null>,
#   "season": <season_year or null>,
#   "event_type": {{
#     "type": <Goal|Card|subst|Var or null>,
#     "detail": <valid detail string or null>
#   }},
#   "referee": <referee_name or null>,
#   "round": <round number or null>,
#   "venue": <venue name or null>,
#   "aggregation": <"count" | "sum" | "avg" | "max" | "min" | "list" | null>
# }}

# Valid event type + detail combinations include:
# Goal → Penalty, Normal Goal, Own Goal, Missed Penalty
# Card → Yellow Card, Red Card, (empty)
# subst → Substitution (covers all numbered substitutions)
# Var → Goal cancelled, Penalty cancelled, Card upgrade, Goal confirmed, Card reviewed, Penalty awarded, Penalty confirmed, Red card cancelled, Goal Disallowed, Goal Disallowed - offside, Goal Disallowed - handball, Goal Disallowed - Foul, (empty)

# Always normalize substitution details to just "Substitution"

# User query: "{query}"
# """
# )

# # === 3. ENTITY EXTRACTION CHAIN ===
# entity_chain = LLMChain(llm=llm, prompt=entity_extraction_prompt, verbose=True)

# def extract_entities(user_query: str) -> dict:
#     try:
#         response = entity_chain.run(query=user_query)
#         return json.loads(response)
#     except Exception as e:
#         print("\u26a0\ufe0f Failed to parse entity JSON:", str(e))
#         return {}

# # === 4. FUZZY MATCHING LOOKUP ===
# def fuzzy_lookup(query_name: str, table: str, column: str, id_column: str, threshold: int = 75) -> int:
#     try:
#         conn = mysql.connector.connect(
#             host=DB_HOST, user=DB_USER, password=DB_PASS, database=DB_NAME, port=DB_PORT
#         )
#         cursor = conn.cursor()
#         cursor.execute(f"SELECT {id_column}, {column} FROM {table}")
#         rows = cursor.fetchall()
#         conn.close()

#         name_dict = {row[1]: row[0] for row in rows if row[1]}
#         filtered_matches = process.extract(
#             query_name, name_dict.keys(), scorer=fuzz.token_sort_ratio, limit=5
#         )
#         filtered_matches = [m for m in filtered_matches if m[1] >= threshold]
#         if filtered_matches:
#             best_match = sorted(filtered_matches, key=lambda x: (x[1], len(x[0])), reverse=True)[0]
#             print(f"✅ Fuzzy matched '{query_name}' → '{best_match[0]}' ({best_match[1]}%)")
#             return name_dict[best_match[0]]
#         else:
#             print(f"⚠️ No good fuzzy match for '{query_name}'")
#             return None
#     except Exception as e:
#         print(f"❌ Error in fuzzy lookup for '{query_name}': {str(e)}")
#         return None

# # === 5. ENTITY -> ID FUNCTIONS ===
# def get_player_id(name: str) -> int:
#     return fuzzy_lookup(name, "base_data_apis.players", "name", "player_id")

# def get_team_id(name: str) -> int:
#     return fuzzy_lookup(name, "base_data_apis.teams", "name", "team_id")

# def get_league_id(name: str) -> int:
#     return fuzzy_lookup(name, "base_data_apis.leagues", "name", "league_id")

# # === 6. VALID EVENT TYPES ===
# VALID_EVENT_TYPES = {
#     ("Goal", "Penalty"), ("Goal", "Normal Goal"), ("Goal", "Own Goal"), ("Goal", "Missed Penalty"),
#     ("Card", "Yellow Card"), ("Card", "Red Card"), ("Card", ""),
#     ("Var", "Goal cancelled"), ("Var", "Penalty cancelled"), ("Var", "Card upgrade"), ("Var", "Goal confirmed"),
#     ("Var", "Card reviewed"), ("Var", "Penalty awarded"), ("Var", "Penalty confirmed"), ("Var", "Red card cancelled"),
#     ("Var", "Goal Disallowed"), ("Var", "Goal Disallowed - offside"), ("Var", "Goal Disallowed - handball"),
#     ("Var", "Goal Disallowed - Foul"), ("Var", ""),
#     ("subst", "Substitution")
# }

# def normalize_event_type_pair(event_type: str, event_detail: str) -> tuple:
#     if event_type == "subst" and event_detail and event_detail.lower().startswith("substitution"):
#         return ("subst", "Substitution")
#     return (event_type or "", event_detail or "")

# # === 7. MAIN AGENT FUNCTION ===
# def resolve_entities_to_ids(user_query: str) -> dict:
#     entities = extract_entities(user_query)
#     resolved = {}

#     if entities.get("player"):
#         resolved["player_id"] = get_player_id(entities["player"])

#     if entities.get("team"):
#         team_id = get_team_id(entities["team"])
#         role = entities.get("team_role", "for")
#         if role == "for":
#             resolved["team_id"] = team_id
#         elif role == "against":
#             resolved["opponent_team_id"] = team_id

#     if entities.get("league"):
#         league_name = entities["league"]
#         resolved["league_id"] = 39 if league_name and league_name.lower() == "premier league" else get_league_id(league_name)

#     if entities.get("season"):
#         try:
#             resolved["season_year"] = int(entities["season"])
#         except:
#             resolved["season_year"] = None

#     if isinstance(entities.get("event_type"), dict):
#         etype = entities["event_type"].get("type")
#         edetail = entities["event_type"].get("detail")

#         if etype in {"Goal", "Card", "Var", "subst"} and (edetail is None or edetail.strip() == ""):
#             resolved["event_type"] = etype
#         else:
#             normalized = normalize_event_type_pair(etype, edetail)
#             if normalized in VALID_EVENT_TYPES:
#                 resolved["event_type"] = normalized[0]
#                 resolved["event_detail"] = normalized[1]
#             else:
#                 print("⚠️ Invalid event type/detail pair:", normalized)

#     if entities.get("aggregation"):
#         resolved["aggregation"] = entities["aggregation"]

#     resolved["raw_entities"] = entities
#     return resolved

# # === 8. QUERY GENERATOR ===
# def build_filtered_event_query(filters: dict) -> str:
#     if filters.get("aggregation") == "count":
#         return "SELECT COUNT(*) FROM (" + build_filtered_event_query({k: v for k, v in filters.items() if k != "aggregation"})[:-1] + ") as count_table;"

#     base_query = """
#     SELECT
#       e.event_id,
#       e.type,
#       p.name AS player_name,
#       e.detail,
#       e.comments,
#       e.minute,
#       e.extra_minute,
#       f.fixture_id,
#       f.referee,
#       f.fixture_date,
#       f.venue_id,
#       f.season_year,
#       f.league_id,
#       f.league_round,
#       f.total_home_goals,
#       f.total_away_goals,
#       e.team_id AS player_team_id,
#       t.name AS player_team_name,
#       CASE
#         WHEN e.team_id = f.home_team_id THEN f.away_team_id
#         ELSE f.home_team_id
#       END AS opponent_team_id,
#       CASE
#         WHEN e.team_id = f.home_team_id THEN (SELECT name FROM base_data_apis.teams WHERE team_id = f.away_team_id)
#         ELSE (SELECT name FROM base_data_apis.teams WHERE team_id = f.home_team_id)
#       END AS opponent_team_name,
#       a.name AS assist_name
#     FROM base_data_apis.fixture_events e
#     JOIN base_data_apis.fixtures f ON e.fixture_id = f.fixture_id
#       AND (e.team_id = f.home_team_id OR e.team_id = f.away_team_id)
#     LEFT JOIN base_data_apis.players p ON e.player_id = p.player_id
#     LEFT JOIN base_data_apis.players a ON e.assist_id = a.player_id
#     LEFT JOIN base_data_apis.teams t ON e.team_id = t.team_id
#     WHERE 1=1
#     """

#     conditions = []

#     if filters.get("player_id"):
#         conditions.append(f"AND e.player_id = {filters['player_id']}")
#     if filters.get("team_id"):
#         conditions.append(f"AND e.team_id = {filters['team_id']}")
#     if filters.get("opponent_team_id"):
#         conditions.append(f"AND (f.home_team_id = {filters['opponent_team_id']} OR f.away_team_id = {filters['opponent_team_id']})")
#         conditions.append(f"AND e.team_id != {filters['opponent_team_id']}")
#     if filters.get("league_id"):
#         conditions.append(f"AND f.league_id = {filters['league_id']}")
#     if filters.get("season_year"):
#         conditions.append(f"AND f.season_year = {filters['season_year']}")
#     if filters.get("event_type"):
#         conditions.append(f"AND e.type = '{filters['event_type']}'")
#     if filters.get("event_detail") and filters["event_detail"].strip():
#         conditions.append(f"AND e.detail = '{filters['event_detail']}'")

#     return base_query + "\n".join(conditions) + ";"

# # === 9. EXECUTOR ===
# def run_sql_query(query: str) -> str:
#     try:
#         conn = mysql.connector.connect(
#             host=DB_HOST, user=DB_USER, password=DB_PASS, database=DB_NAME, port=DB_PORT
#         )
#         cursor = conn.cursor()
#         cursor.execute(query)
#         rows = cursor.fetchall()
#         columns = [desc[0] for desc in cursor.description]
#         conn.close()

#         if not rows:
#             return "No results found."

#         # If COUNT aggregation
#         if "COUNT(*)" in query:
#             return f"Total count: {rows[0][0]}"

#         return "Columns: " + str(columns) + "\n" + "\n".join([str(row) for row in rows])
#     except Exception as e:
#         return f"❌ Error running query: {str(e)}"

# # === 10. NATURAL LANGUAGE RESPONSE GENERATOR ===
# def generate_natural_response(user_query: str, filters: dict, result: str) -> str:
#     response_prompt = PromptTemplate(
#         input_variables=["query", "filters", "result"],
#         template="""
# You are an intelligent assistant that interprets football analytics data.

# User asked: "{query}"

# Based on the following extracted filters: {filters}

# And the result from the database: {result}

# Generate a natural language answer that sounds conversational and directly answers the user's question. Don't just repeat the filters; answer as if you understood the question. Keep it simple and fluent.
# """
#     )
#     response_chain = LLMChain(llm=llm, prompt=response_prompt)
#     return response_chain.run(query=user_query, filters=json.dumps(filters), result=result)

# # === 11. DEMO RUN ===
# if __name__ == "__main__":
#     print("\n⚽ Welcome to the Football Analytics Assistant!")
#     print("Ask any football-related question (e.g., 'How many goals did Haaland score in 2023?')")
#     print("Type 'exit' to quit.\n")

#     while True:
#         user_query = input("Enter your question (or 'exit'): ").strip()
#         if user_query.lower() in {"exit", "quit"}:
#             print("\n👋 Thanks for using the assistant. Goodbye!\n")
#             break

#         print(f"\n>>> You asked: {user_query}\n")

#         # Step 1: Extract and resolve entities
#         filters = resolve_entities_to_ids(user_query)

#         # Step 2: Build SQL query
#         sql = build_filtered_event_query(filters)
#         print("📄 Final SQL Query:")
#         print(sql)

#         # Step 3: Run the query
#         result = run_sql_query(sql)
#         print("\n📊 Query Result:")
#         print(result)

#         # Step 4: Generate a natural language answer
#         answer = generate_natural_response(user_query, filters, result)
#         print(f"\n🗣️ {answer}\n")
#         print("— Done.\n")



⚽ Welcome to the Football Analytics Assistant!
Ask any football-related question (e.g., 'How many goals did Haaland score in 2023?')
Type 'exit' to quit.

Enter your question (or 'exit'): exit

👋 Thanks for using the assistant. Goodbye!



In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from sentence_transformers import SentenceTransformer
import mysql.connector, json, faiss, os
from rapidfuzz import process, fuzz



# === 1. SETUP LLM + EMBEDDINGS ===
llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1", api_key=NVIDIA_API_KEY)
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

# === 2. FAISS Index Builder ===
import pickle

FAISS_DIR = "faiss_data"
os.makedirs(FAISS_DIR, exist_ok=True)

def load_table_data(table, id_col, name_col):
    conn = mysql.connector.connect(host=DB_HOST, user=DB_USER, password=DB_PASS, database=DB_NAME, port=DB_PORT)
    cur = conn.cursor()
    cur.execute(f"SELECT {id_col}, {name_col} FROM {table}")
    rows = cur.fetchall()
    conn.close()
    return rows

def save_faiss_index(index, names, ids, basename):
    faiss.write_index(index, f"{FAISS_DIR}/{basename}.index")
    with open(f"{FAISS_DIR}/{basename}_meta.pkl", "wb") as f:
        pickle.dump({"names": names, "ids": ids}, f)

def load_faiss_index(basename):
    try:
        index = faiss.read_index(f"{FAISS_DIR}/{basename}.index")
        with open(f"{FAISS_DIR}/{basename}_meta.pkl", "rb") as f:
            meta = pickle.load(f)
        return index, meta["names"], meta["ids"]
    except:
        return None, None, None

def build_or_load_index(basename, table, id_col, name_col):
    index, names, ids = load_faiss_index(basename)
    if index is not None:
        print(f"✅ Loaded FAISS index for {basename}")
        return index, names, ids
    print(f"🔁 Building FAISS index for {basename}...")
    rows = load_table_data(table, id_col, name_col)
    names = [r[1] for r in rows]
    ids = [r[0] for r in rows]
    embs = embed_model.encode(names, normalize_embeddings=True)
    index = faiss.IndexFlatIP(embs.shape[1])
    index.add(embs)
    save_faiss_index(index, names, ids, basename)
    return index, names, ids

# Load or build FAISS indexes
player_index, player_names, player_ids = build_or_load_index("players", "base_data_apis.players", "player_id", "name")
team_index, team_names, team_ids       = build_or_load_index("teams",   "base_data_apis.teams",   "team_id",   "name")
league_index, league_names, league_ids = build_or_load_index("leagues", "base_data_apis.leagues", "league_id", "name")

# === 3. ENTITY EXTRACTION ===
entity_prompt = PromptTemplate(
    input_variables=["query"],
    template="""
You are a football data assistant. Extract entities from the query in JSON format:

{{
  "player": <name or null>,
  "team": <name or null>,
  "team_role": "for" | "against" | null,
  "league": <name or null>,
  "season": <year or null>,
  "event_type": {{
    "type": <Goal|Card|subst|Var or null>,
    "detail": <string or null>
  }},
  "aggregation": <"count"|"sum"|"avg"|"list"|null>
}}

Query: "{query}"
"""
)
entity_chain = LLMChain(llm=llm, prompt=entity_prompt, verbose=False)

def extract_entities(query):
    try: return json.loads(entity_chain.run(query=query))
    except: return {}

# === 4. VALID EVENT TYPES ===
VALID_EVENT_TYPES = {
    ("Goal", "Penalty"), ("Goal", "Normal Goal"), ("Goal", "Own Goal"), ("Goal", "Missed Penalty"),
    ("Card", "Yellow Card"), ("Card", "Red Card"), ("Card", ""),
    ("Var", "Goal cancelled"), ("Var", "Penalty cancelled"), ("Var", "Goal confirmed"), ("Var", ""),
    ("subst", "Substitution")
}
def normalize_event(etype, edetail):
    if etype == "subst": return ("subst", "Substitution")
    return (etype or "", edetail or "")

# === 5. RESOLVE ENTITIES TO IDs ===
def resolve_entities_to_ids(user_query):
    e = extract_entities(user_query)
    r = {}

    if e.get("player"):
        r["player_id"] = faiss_lookup(e["player"], player_index, player_names, player_ids) or fuzzy_lookup(e["player"], "base_data_apis.players", "name", "player_id")
    if e.get("team"):
        tid = faiss_lookup(e["team"], team_index, team_names, team_ids) or fuzzy_lookup(e["team"], "base_data_apis.teams", "name", "team_id")
        if e.get("team_role", "for") == "for": r["team_id"] = tid
        else: r["opponent_team_id"] = tid
    if e.get("league"):
        r["league_id"] = faiss_lookup(e["league"], league_index, league_names, league_ids) or fuzzy_lookup(e["league"], "base_data_apis.leagues", "name", "league_id")
    if e.get("season"):
        try: r["season_year"] = int(e["season"])
        except: pass
    if isinstance(e.get("event_type"), dict):
        etype, edetail = normalize_event(e["event_type"].get("type"), e["event_type"].get("detail"))
        if (etype, edetail) in VALID_EVENT_TYPES:
            r["event_type"] = etype
            r["event_detail"] = edetail
    if e.get("aggregation"):
        r["aggregation"] = e["aggregation"]

    r["raw_entities"] = e
    return r

# === 6. SQL QUERY BUILDER ===
def build_sql(filters):
    if filters.get("aggregation") == "count":
        return "SELECT COUNT(*) FROM (" + build_sql({k: v for k, v in filters.items() if k != "aggregation"})[:-1] + ") AS count_sub;"
    base = """
    SELECT e.event_id, e.type, e.detail, e.minute, p.name AS player_name, t.name AS team_name
    FROM base_data_apis.fixture_events e
    JOIN base_data_apis.fixtures f ON f.fixture_id = e.fixture_id
    LEFT JOIN base_data_apis.players p ON e.player_id = p.player_id
    LEFT JOIN base_data_apis.teams t ON e.team_id = t.team_id
    WHERE 1=1
    """
    conditions = []
    if filters.get("player_id"): conditions.append(f"AND e.player_id = {filters['player_id']}")
    if filters.get("team_id"): conditions.append(f"AND e.team_id = {filters['team_id']}")
    if filters.get("opponent_team_id"):
        conditions.append(f"AND (f.home_team_id = {filters['opponent_team_id']} OR f.away_team_id = {filters['opponent_team_id']})")
        conditions.append(f"AND e.team_id != {filters['opponent_team_id']}")
    if filters.get("league_id"): conditions.append(f"AND f.league_id = {filters['league_id']}")
    if filters.get("season_year"): conditions.append(f"AND f.season_year = {filters['season_year']}")
    if filters.get("event_type"): conditions.append(f"AND e.type = '{filters['event_type']}'")
    if filters.get("event_detail"): conditions.append(f"AND e.detail = '{filters['event_detail']}'")
    return base + "\n".join(conditions) + ";"

# === 7. EXECUTE SQL ===
def run_sql(sql):
    try:
        conn = mysql.connector.connect(host=DB_HOST, user=DB_USER, password=DB_PASS, database=DB_NAME, port=DB_PORT)
        cur = conn.cursor(); cur.execute(sql)
        rows = cur.fetchall(); cols = [d[0] for d in cur.description]
        conn.close()
        if "COUNT(*)" in sql: return f"Total count: {rows[0][0]}"
        return " | ".join(cols) + "\n" + "\n".join([str(r) for r in rows])
    except Exception as e:
        return f"❌ SQL error: {e}"

# === 8. NATURAL LANGUAGE RESPONSE ===
response_prompt = PromptTemplate(
    input_variables=["query", "filters", "result", "raw_ent"],
    template="""
You are a football analytics assistant. Based on:
- User query: "{query}"
- Extracted: {raw_ent}
- Filters applied: {filters}
- SQL result: {result}

Give a simple, friendly answer explaining the result.
"""
)
response_chain = LLMChain(llm=llm, prompt=response_prompt)

# === 9. MAIN LOOP ===
if __name__ == "__main__":
    print("⚽ Ask your football question (type 'exit' to quit)")
    while True:
        user_query = input(">>> ").strip()
        if user_query.lower() == "exit":
            print("👋 Bye!")
            break
        filters = resolve_entities_to_ids(user_query)
        sql = build_sql(filters)
        print(f"\n📄 SQL Query:\n{sql}")
        result = run_sql(sql)
        print(f"\n📊 Result:\n{result}")
        answer = response_chain.run(query=user_query, filters=json.dumps(filters), result=result, raw_ent=json.dumps(filters.get("raw_entities", {})))
        print(f"\n🗣️ {answer}\n{'—'*50}\n")


✅ Loaded FAISS index for players
✅ Loaded FAISS index for teams
✅ Loaded FAISS index for leagues
⚽ Ask your football question (type 'exit' to quit)
❌ Entity extraction failed: Invalid \escape: line 4 column 8 (char 45)

📄 SQL Query:

    SELECT e.event_id, e.type, e.detail, e.minute, p.name AS player_name, t.name AS team_name
    FROM base_data_apis.fixture_events e
    JOIN base_data_apis.fixtures f ON f.fixture_id = e.fixture_id
    LEFT JOIN base_data_apis.players p ON e.player_id = p.player_id
    LEFT JOIN base_data_apis.teams t ON e.team_id = t.team_id
    WHERE 1=1
    ;
Buffered data was truncated after reaching the output size limit.

In [ ]:
# # ✅ LANGGRAPH MODULAR AGENT IMPLEMENTATION

# from langchain.chains import LLMChain
# from langchain.prompts import PromptTemplate
# from langchain_nvidia_ai_endpoints import ChatNVIDIA
# import mysql.connector
# import os
# import json
# from rapidfuzz import process, fuzz
# from langgraph.graph import StateGraph, END

# # === 1. SETUP LLM ===
# llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1", api_key=NVIDIA_API_KEY)

# # === 2. ENTITY EXTRACTION MODULE ===
# def extract_entities_module(state: dict) -> dict:
#     prompt = PromptTemplate(
#         input_variables=["query"],
#         template="""
# You are a football data assistant. Extract and classify entities from the user's query into this JSON:

# {
#   "player": <player_name or null>,
#   "team": <full team name (e.g., Manchester City) or null>,
#   "team_role": "for" | "against" | null,
#   "league": <league_name or null>,
#   "season": <season_year or null>,
#   "event_type": {
#     "type": <Goal|Card|subst|Var or null>,
#     "detail": <valid detail string or null>
#   },
#   "referee": <referee_name or null>,
#   "round": <round number or null>,
#   "venue": <venue name or null>
# }

# User query: "{query}"
# """
#     )
#     chain = LLMChain(llm=llm, prompt=prompt)
#     try:
#         response = chain.run(query=state["query"])
#         state["entities"] = json.loads(response)
#     except Exception as e:
#         state["error"] = f"Failed to parse entities: {e}"
#     return state

# # === 3. ID RESOLUTION MODULE ===
# def resolve_ids_module(state: dict) -> dict:
#     entities = state.get("entities", {})
#     resolved = {}

#     def fuzzy_lookup(query_name, table, column, id_column):
#         try:
#             conn = mysql.connector.connect(host=DB_HOST, user=DB_USER, password=DB_PASS, database=DB_NAME, port=DB_PORT)
#             cursor = conn.cursor()
#             cursor.execute(f"SELECT {id_column}, {column} FROM {table}")
#             rows = cursor.fetchall()
#             conn.close()
#             name_dict = {row[1]: row[0] for row in rows if row[1]}
#             match = process.extractOne(query_name, name_dict.keys(), scorer=fuzz.token_sort_ratio)
#             return name_dict[match[0]] if match and match[1] > 75 else None
#         except:
#             return None

#     if entities.get("player"):
#         resolved["player_id"] = fuzzy_lookup(entities["player"], "base_data_apis.players", "name", "player_id")

#     if entities.get("team"):
#         team_id = fuzzy_lookup(entities["team"], "base_data_apis.teams", "name", "team_id")
#         role = entities.get("team_role", "for")
#         if role == "for":
#             resolved["team_id"] = team_id
#         else:
#             resolved["opponent_team_id"] = team_id

#     if entities.get("league"):
#         if entities["league"].lower() == "premier league":
#             resolved["league_id"] = 39
#         else:
#             resolved["league_id"] = fuzzy_lookup(entities["league"], "base_data_apis.leagues", "name", "league_id")

#     if entities.get("season"):
#         try:
#             resolved["season_year"] = int(entities["season"])
#         except:
#             pass

#     if isinstance(entities.get("event_type"), dict):
#         etype = entities["event_type"].get("type")
#         edetail = entities["event_type"].get("detail")
#         resolved["event_type"] = etype
#         if edetail: resolved["event_detail"] = edetail

#     if entities.get("aggregation_hint"):
#         resolved["aggregation"] = entities["aggregation_hint"]
#     elif "how many" in state["query"].lower():
#         resolved["aggregation"] = "count"

#     state["filters"] = resolved
#     return state

# # === 4. SQL GENERATION MODULE ===
# def sql_generation_module(state: dict) -> dict:
#     f = state.get("filters", {})
#     base = """
#     SELECT
#       e.event_id,
#       e.type,
#       p.name AS player_name,
#       e.detail,
#       e.minute,
#       f.fixture_date,
#       f.season_year,
#       f.league_id,
#       t.name AS player_team_name,
#       CASE
#         WHEN e.team_id = f.home_team_id THEN (SELECT name FROM base_data_apis.teams WHERE team_id = f.away_team_id)
#         ELSE (SELECT name FROM base_data_apis.teams WHERE team_id = f.home_team_id)
#       END AS opponent_team_name
#     FROM base_data_apis.fixture_events e
#     JOIN base_data_apis.fixtures f ON e.fixture_id = f.fixture_id
#     LEFT JOIN base_data_apis.players p ON e.player_id = p.player_id
#     LEFT JOIN base_data_apis.teams t ON e.team_id = t.team_id
#     WHERE 1=1
#     """
#     conditions = []
#     if f.get("player_id"): conditions.append(f"AND e.player_id = {f['player_id']}")
#     if f.get("team_id"): conditions.append(f"AND e.team_id = {f['team_id']}")
#     if f.get("opponent_team_id"):
#         conditions.append(f"AND (f.home_team_id = {f['opponent_team_id']} OR f.away_team_id = {f['opponent_team_id']})")
#         conditions.append(f"AND e.team_id != {f['opponent_team_id']}")
#     if f.get("league_id"): conditions.append(f"AND f.league_id = {f['league_id']}")
#     if f.get("season_year"): conditions.append(f"AND f.season_year = {f['season_year']}")
#     if f.get("event_type"): conditions.append(f"AND e.type = '{f['event_type']}'")
#     if f.get("event_detail"): conditions.append(f"AND e.detail = '{f['event_detail']}'")

#     full_query = base + "\n".join(conditions)
#     state["sql"] = f"SELECT COUNT(*) FROM ({full_query}) AS sub;" if f.get("aggregation") == "count" else full_query + ";"
#     return state

# # === 5. SQL EXECUTION MODULE ===
# def sql_execution_module(state: dict) -> dict:
#     try:
#         conn = mysql.connector.connect(host=DB_HOST, user=DB_USER, password=DB_PASS, database=DB_NAME, port=DB_PORT)
#         cursor = conn.cursor()
#         cursor.execute(state["sql"])
#         rows = cursor.fetchall()
#         conn.close()
#         state["result"] = rows
#     except Exception as e:
#         state["error"] = f"SQL error: {e}"
#     return state

# # === 6. RESPONSE GENERATOR MODULE ===
# def response_module(state: dict) -> dict:
#     if "result" in state and state.get("filters", {}).get("aggregation") == "count":
#         player = state["entities"].get("player", "The player")
#         team = state["entities"].get("team", "the team")
#         season = state["entities"].get("season", "all seasons")
#         count = state["result"][0][0]
#         state["final_answer"] = f"{player} scored {count} goals for {team} in {season}."
#     elif "result" in state:
#         state["final_answer"] = str(state["result"])
#     return state

# # === 7. LANGGRAPH FLOW ===
# graph = StateGraph()
# graph.add_node("entity_extraction", extract_entities_module)
# graph.add_node("id_resolution", resolve_ids_module)
# graph.add_node("sql_generation", sql_generation_module)
# graph.add_node("sql_execution", sql_execution_module)
# graph.add_node("response", response_module)

# graph.set_entry_point("entity_extraction")
# graph.add_edge("entity_extraction", "id_resolution")
# graph.add_edge("id_resolution", "sql_generation")
# graph.add_edge("sql_generation", "sql_execution")
# graph.add_edge("sql_execution", "response")
# graph.add_edge("response", END)

# app = graph.compile()

# # === 8. DEMO ===
# if __name__ == "__main__":
#     user_query = "How many goals did Rashford score for United in Premier League in 2023?"
#     result = app.invoke({"query": user_query})
#     print(result.get("final_answer", "No answer produced."))
